In [1]:
import onnx
import numpy as np
from builder import *
import onnxruntime as ort

In [2]:
def maxpool_session():
    input = Placeholder()
    b = Exporter()
    b.add_graph_input('input', input, np.float32, [None, None, None, None])
    b.add_graph_output('output', MaxPool(input, pads=[1, 1, 1, 1], kernel_shape=[2,2]), np.float32, [None, None, None, None])
    return ort.InferenceSession(b.export("MaxPool").SerializeToString())

msess = maxpool_session()

image = -1e5*np.ones([1, 3, 3, 1], dtype=np.float32)
msess.run(['output'], {'input':ort.OrtValue.ortvalue_from_numpy(image)})

[array([[[[-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.]],
 
         [[-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.]],
 
         [[-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.],
          [-100000., -100000.]]]], dtype=float32)]

In [3]:
def add_session():
    X = Placeholder()
    Y = Placeholder()
    b = Exporter()
    b.add_graph_input('x', X, np.float32, [None])
    b.add_graph_input('y', Y, np.float32, [None])
    b.add_graph_output('Z', X+Y, np.float32)
    return ort.InferenceSession(b.export('Add test').SerializeToString())

asess = add_session()

x1 = np.asarray([1,2,4,8], dtype=np.float32)
y1 = np.asarray([1,3,9,27], dtype=np.float32)
z1 = asess.run(['Z'], {'x':ort.OrtValue.ortvalue_from_numpy(x1), 'y':ort.OrtValue.ortvalue_from_numpy(y1)})

x2 = np.asarray([i for i in range(10)], dtype=np.float32)
y2 = np.asarray([i*i for i in range(10)], dtype=np.float32)
z2 = asess.run(['Z'], {'x':ort.OrtValue.ortvalue_from_numpy(x2), 'y':ort.OrtValue.ortvalue_from_numpy(y2)})

In [4]:
def resize_session(inshape=[2, 2, 2], outshape=[5, 5, 5], coordinate_transformation_mode="pytorch_half_pixel", mode="linear", **kwargs):
    P = Placeholder()
    sizes = Constant(np.asarray(outshape, dtype=np.int64))
    b = Exporter()
    b.add_graph_input('input', P, np.float32, inshape)
    b.add_graph_output('output', Resize(P, sizes=sizes, coordinate_transformation_mode=coordinate_transformation_mode,
                       mode=mode, **kwargs), np.float32, [None]*len(inshape))
    return ort.InferenceSession(b.export('test').SerializeToString())


def resize_test(inshape, outshape, **kwargs):
    rsess = resize_session(inshape=inshape, outshape=outshape, **kwargs)
    input = np.asarray([(i*i+1) for i in range(np.prod(inshape))],
                       dtype=np.float32).reshape(inshape)
    print("/* Generated by ONNX Runtime */")
    #print(f'Input:\n{input}')
    ortoutput = rsess.run(['output'], {'input': ort.OrtValue.ortvalue_from_numpy(input)})
    #print(ortoutput)
    output = np.asarray(ortoutput)
    #print(f'Output:\n{output}')
    return input, output

ctms = ["half_pixel", "pytorch_half_pixel", "align_corners", "asymmetric"]
nearmodes = ["round_prefer_floor", "round_prefer_ceil", "floor", "ceil"]


def cvec(v):
    comma = ""
    result = "{"
    for elt in v:
        result += comma
        comma = ", "
        result += str(elt)
    result += "}"
    return result


In [2]:


def run_resize_nearest_tests():
    for coordinate_transformation_mode in ["align_corners", "asymmetric"]: #ctms:
        for nearmode in nearmodes:
            for insize in [1, 2, 3, 5]:
                inShape = [1] + [insize]*2 + [1]
                for outsize in [1, 2, 3, 5, 16]:
                    outShape = [1] + [outsize]*2 + [1]
                    input, output = resize_test(inShape, outShape, mode="nearest", nearest_mode=nearmode,
                                                coordinate_transformation_mode=coordinate_transformation_mode)
                    print(
                        f"TEST_P(OperatorTest, ResizeNearestCase_{insize}_{outsize}_{coordinate_transformation_mode}_{nearmode}){' {'}")
                    print(f"  CHECK_IF_ENABLED();")
                    print(f"  testResizeNearestCase(bindings_, mod_, F_, EE_,")
                    print(
                        f"                        {cvec(inShape)}, {cvec(input.flatten())},")
                    print(
                        f"                        {cvec(outShape)}, {cvec(output.flatten())},")
                    print(
                        f"                        ResizeCoordinateTransformationMode::{coordinate_transformation_mode.upper()},")
                    print(
                        f"                        ResizeNearestMode::{nearmode.upper()}, true);")
                    print(f"{'}'}")
                    print()


run_resize_nearest_tests()


NameError: name 'nearmodes' is not defined

In [1]:
def run_resize_linear_tests():
    for coordinate_transformation_mode in ctms:
        for insize in [4, 5, 8]:
            inShape = [1] + [insize]*2 + [1]
            for outsize in [1, 5, 16]:
                outShape = [1] + [outsize]*2 + [1]
                input, output = resize_test(inShape, outShape, mode="linear",
                                            coordinate_transformation_mode=coordinate_transformation_mode)
                print(
                    f"TEST_P(OperatorTest, ResizeLinearCase_{insize}_{outsize}_{coordinate_transformation_mode}){' {'}")
                print(f"  CHECK_IF_ENABLED();")
                print(f"  testResizeLinearCase(bindings_, mod_, F_, EE_,")
                print(
                    f"                        {cvec(inShape)}, {cvec(input.flatten())},")
                print(
                    f"                        {cvec(outShape)}, {cvec(output.flatten())},")
                print(
                    f"                        ResizeCoordinateTransformationMode::{coordinate_transformation_mode.upper()},")
                print(
                    f"                        true);")
                print(f"{'}'}")
                print()


run_resize_linear_tests()


NameError: name 'ctms' is not defined